# Group Work

## 1 Background

## 2 Data Pre-process
### 2.1 Data Cleaning
下面2.1.1, 2.1.2标题内容是我一开始写的，你们按需要自己改吧

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

#### 2.1.1 Parameterising Files

In [2]:
# Set download URL
ymd  = '20250615'
city = 'London'
host = 'https://orca.casa.ucl.ac.uk'
url  = f'{host}/~jreades/data/{ymd}-{city}-listings.csv.gz'
print(url)

https://orca.casa.ucl.ac.uk/~jreades/data/20250615-London-listings.csv.gz


#### 2.1.2 Reading CSV with Pandas

In [5]:
df_raw = pd.read_csv(url, compression='gzip', low_memory=False)
print(f"Data frame is {df_raw.shape[0]:,} x {df_raw.shape[1]}")

Data frame is 96,651 x 79


### 2.2 Overview of the London Airbnb Market

### 2.3 Spatial Distribution of Airbnb Listings

## 3 Is Airbnb in London out of control?

### 3.1 How to define "out of control"
hard to define directly →
cause it is used to describe the whole London area, which is large. And there is no related standards for reference (e.g. 30% of Airbnb houses London are priced above 1000$/night can be treated as out of control)→
so we introduce the concept of "misuse", which is much easier to implement, because it describes whether a samller unit: a single property is legally operated (90 days etc) →
we use how many properties are misused to help us find out whther London Airbnb Marker is out of control

### 3.2 How many properties are misused?

### 3.3 The distribution patterns of misused properties

### 3.4 The influence of Airbnb misuses
for housing rent? or other factors

In [ ]:
回归？要做吗？

### 3.5 Conclusion: **Airbnb in London is out of control !**

## 4 How many professional landlords are there?

### 4.1 How to define professional landlords
The landlords who own 2 or more entire houses and rent them out on Airbnb for more than 90 days a year.

### 4.2 How many professional landlords in London

### 4.3 Where are these professional landlords distributed

### 4.4 Conclusion


## 5 How many properties would be affected by the opposition’s proposal?

### 5.1 What kind of properties would be affected?
The properties would be affected: **in hot-sopt districts, professional hosts, entire house**

### 5.2 How many properties would be affected in London

### 5.3 Conclusion

## 6 What are the likely pros and cons of the opposition’s proposal?

### 6.1 Pros of the proposal

#### 6.1.1 For the Mayor

#### 6.1.2 For the residents

#### 6.1.3 For the city

### 6.2 Cons of the proposal
same structure as pros

#### 6.2.1 For the Mayor

#### 6.2.2 For the residents

#### 6.2.3 For the city

## 7 Can the story be reframed as a positive one about social mobility or housing opportunity?

### 7.1 

### 7.2 

### 7.3 

## 8 Reference

[1]